# Challenge Set 9
## Part II: Baseball Data

*Introductory - Intermediate level SQL*

--

Please complete this exercise via SQLalchemy and Jupyter notebook.

We will be working with the Lahman baseball data we uploaded to your AWS instance in class. 


1. What was the total spent on salaries by each team, each year?

2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

3. Who has played the most all star games?

4. Which school has generated the most distinct players? *Hint:* Create new table from 'CollegePlaying.csv'.

5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

6. What is the distribution of debut months? *Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

7. What is the effect of table join order on mean salary for the players listed in the main (master) table? *Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [61]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column, String, INTEGER, VARCHAR, TEXT
import postgres_copy

import pandas as pd

In [49]:
cnx = create_engine('postgresql://cneiderer@54.159.198.159:5432/baseball')

In [50]:
cnx.table_names()

['allstarfull', 'salaries', 'schools', 'fielding']

In [8]:
df = {}
for t in cnx.table_names():
    df[t] = pd.read_sql_query('SELECT * FROM ' + t, cnx)

In [9]:
df['allstarfull'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4912 entries, 0 to 4911
Data columns (total 8 columns):
playerid       4912 non-null object
yearid         4912 non-null int64
gamenum        4912 non-null object
gameid         4863 non-null object
teamid         4912 non-null object
lgid           4912 non-null object
gp             4875 non-null object
startingpos    1540 non-null object
dtypes: int64(1), object(7)
memory usage: 307.1+ KB


In [10]:
df['salaries'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23956 entries, 0 to 23955
Data columns (total 5 columns):
yearid      23956 non-null int64
teamid      23956 non-null object
lgid        23956 non-null object
playerid    23956 non-null object
salary      23956 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 935.9+ KB


In [11]:
df['schools'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 5 columns):
schoolid       749 non-null object
schoolname     749 non-null object
schoolcity     749 non-null object
schoolstate    749 non-null object
schoolnick     744 non-null object
dtypes: object(5)
memory usage: 29.3+ KB


### What was the total spent on salaries by each team, each year?

In [14]:
df['salaries'].groupby(['teamid', 'yearid']).sum()

salary
teamid yearid             
ANA    1997     31135472.0
       1998     41281000.0
       1999     55388166.0
       2000     51464167.0
       2001     47535167.0
       2002     61721667.0
       2003     79031667.0
       2004    100534667.0
ARI    1998     32347000.0
       1999     68703999.0
       2000     81027833.0
       2001     85082999.0
       2002    102819999.0
       2003     80657000.0
       2004     69780750.0
       2005     62329166.0
       2006     59684226.0
       2007     52067546.0
       2008     66202712.0
       2009     73115666.0
       2010     60718166.0
       2011     53639833.0
       2012     73804833.0
       2013     90132000.0
ATL    1985     14807000.0
       1986     17102786.0
       1987     16544560.0
       1988     12728174.0
       1989     11112334.0
       1990     14555501.0
...                    ...
TOR    1993     47279166.0
       1994     43433668.0
       1995     50590000.0
       1996     29555083.0
       1997     47079833.0
       1998     51376000.0
       1999     45444333.0
       2000     44838332.0
       2001     76895999.0
       2002     76864333.0
       2003     51269000.0
       2004     50017000.0
       2005     45719500.0
       2006     71365000.0
       2007     81942800.0
       2008     97793900.0
       2009     80538300.0
       2010     62234000.0
       2011     62567800.0
       2012     75009200.0
       2013    126288100.0
WAS    2005     48581500.0
       2006     63143000.0
       2007     36947500.0
       2008     54961000.0
       2009     59928000.0
       2010     61400000.0
       2011     63856928.0
       2012     80855143.0
       2013    113703270.0

[828 rows x 1 columns]

### What is the first and last year played for each player? 
*Hint:* Create a new table from 'Fielding.csv'.

In [68]:
cnx.execute( \
    'CREATE TABLE IF NOT EXISTS Fielding ( \
     playerID varchar(9) NOT NULL, \
     yearID int NOT NULL, \
     stint int NOT NULL, \
     lgID varchar(9) NOT NULL, \
     POS varchar(2) NOT NULL, \
     G int NOT NULL, \
     GS int DEFAULT NULL, \
     InnOuts int DEFAULT NULL, \
     PO int DEFAULT NOT NULL, \
     A int DEFAULT NOT NULL, \
     E int DEFAULT NOT NULL, \
     DP int DEFAULT NOT NULL, \
     PB int DEFAULT NULL, \
     WP int DEFAULT NULL, \
     SB int DEFAULT NULL, \
     CS int DEFAULT NULL, \
     ZR int DEFAULT NULL, \
     PRIMARY KEY (yearID,teamID,lgID,playerID)', 
           cnx);

ProgrammingError: (psycopg2.ProgrammingError) syntax error at or near "NOT"
LINE 1: ...    InnOuts int DEFAULT NULL,      PO int DEFAULT NOT NULL, ...
                                                             ^
 [SQL: 'CREATE TABLE IF NOT EXISTS Fielding (      playerID varchar(9) NOT NULL,      yearID int NOT NULL,      stint int NOT NULL,      lgID varchar(9) NOT NULL,      POS varchar(2) NOT NULL,      G int NOT NULL,      GS int DEFAULT NULL,      InnOuts int DEFAULT NULL,      PO int DEFAULT NOT NULL,      A int DEFAULT NOT NULL,      E int DEFAULT NOT NULL,      DP int DEFAULT NOT NULL,      PB int DEFAULT NULL,      WP int DEFAULT NULL,      SB int DEFAULT NULL,      CS int DEFAULT NULL,      ZR int DEFAULT NULL,      PRIMARY KEY (yearID,teamID,lgID,playerID)'] [parameters: (Engine(postgresql://cneiderer@54.159.198.159:5432/baseball),)]

In [63]:
# with open('/home/cneiderer/baseballdata/Fielding.csv') as f:
#     postgres_copy.copy_from(f, fielding, cnx)

cnx.execute("COPY fielding FROM '/home/cneiderer/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;")


IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "fielding_pkey"
DETAIL:  Key ("playerID", "yearID", "teamID", "lgID")=(ansonca01, 1871, RC1, NA) already exists.
CONTEXT:  COPY fielding, line 27
 [SQL: "COPY fielding FROM '/home/cneiderer/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;"]

### Who has played the most all star games?

In [31]:
df['allstarfull']['gp'] = df['allstarfull']['gp'].astype(float)

In [33]:
df['allstarfull'][['playerid', 'gp']].groupby('playerid').sum().idxmax(axis=0)[0]

'aaronha01'

### Which school has generated the most distinct players? 
*Hint:* Create new table from 'CollegePlaying.csv'.

In [51]:
table = Table('schoolsplayers', MetaData(),
              Column('playerID', VARCHAR(20), primary_key=True),
              Column('schoolID', VARCHAR(20), primary_key=True),
              Column('yearMin', INTEGER),
              Column('yearMax', INTEGER))

table.create(cnx)

### Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. 
*Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

In [55]:
table = Table('master', MetaData(),
              Column('playerID', VARCHAR(20), primary_key=True),
              Column('birthYear', INTEGER, primary_key=True),
              Column('birthMonth', INTEGER, primary_key=True),
              Column('birthDay', INTEGER, primary_key=True),
              Column('birthCountry', TEXT),
              Column('birthState', TEXT),
              Column('birthCity', TEXT),
              Column('deathYear', INTEGER),
              Column('deathMonth', INTEGER),
              Column('deathDay', INTEGER),
              Column('deathCountry', TEXT),
              Column('deathState', TEXT),
              Column('deathCity', TEXT),
              Column('nameFirst', TEXT),
              Column('nameLast', TEXT),
              Column('nameGiven', TEXT),
              Column('weight', INTEGER),
              Column('height', INTEGER),
              Column('bats', VARCHAR(1)),
              Column('throws', VARCHAR(1)),
              Column('debut', VARCHAR(10)),
              Column('finalGame', VARCHAR(10)),
              Column('retroID', VARCHAR(20)),
              Column('bbrefID', VARCHAR(20)))

table.create(cnx)

### What is the distribution of debut months? 
*Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

### What is the effect of table join order on mean salary for the players listed in the main (master) table? 
*Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.